In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression as LR
from statsmodels.formula.api import ols
import statsmodels.api as sm
import scipy.stats as stats
import numbers

## Modelling Function
Takes in the dataframe and optional transform and multicollinearity arguments    
Checks whether to send the data to a transform function   
Selects for x_cols the features from data that are numeric, and not the price   
##### Loop

**1. Remove features with high p-values:**  
- creates the formula using x_cols     
- creates the model using formula   
- creates a dataframe, pv, of the model's p-values  
- creates a new x_cols list from the features in pv with low p-values   

**2. Handle multicollinearity:**     
- Checks whether a function has been passed for handling multicollinearity and calls accordingly    
    
         
            
            
Checks whether the length of x_cols has changed. Loop runs again if it has.     
    
    
When x_cols no longer changes, the final model and x_cols are returned





In [248]:
def modelling(data, transform=None, multicoll=None, alpha=0.05, 
              multicollinearity_threshold = 0.7):
    
    multicollinearity_threshold = 0.7
    
    if transform:
        data = transform(data)
        
    outcome = 'price'
    x_cols = [col for col in (data.drop([outcome], axis=1).columns) 
            if isinstance(data[col][0], numbers.Number)]
    
    while True:
        predictors = '+'.join(x_cols)
        f = outcome + '~' + predictors
        model = ols(formula=f, data=data_train).fit()
        pv = pd.DataFrame(model.pvalues).drop('Intercept')
        pv.rename(columns={0:'p_value'}, inplace=True)
        x_cols = list(pv[pv.p_value <= alpha].index)
        
        if multicoll:
            x_cols = multicoll_remove(data,x_cols, multicollinearity_threshold)
        
        
        
        if len(x_cols) == len(pv):
            break
        
        
    return model, x_cols

## Multicollinearity Function: Remove    
The first multicollinearity function removes a feature from each pair with high multicollinearity    
This function takes in thedata, x_cols and the threshold for removing a feature

- create a dataframe of the correlation of the features in x_cols   
- transform this to get a list of pairs of features with high multicollinearity
- for each pair: check whether they are the same feature or if one of the features has already been listed to be removed; if so, continue to next pair 
- otherwise add the feature with the higher p-value to a list    
- remove the features in this list from x_cols    


return x_cols     




In [249]:
def multicoll_remove(data, x_cols, multicollinearity_threshold):
    corr = data[x_cols].corr().abs().stack().reset_index().sort_values(0,
                                                            ascending = False)
    corr['pairs'] = list(zip(corr.level_0, corr.level_1))
    corr = corr.set_index('pairs').drop(['level_0', 'level_1'], axis=1)
    corr.columns = ['cc']
    corr = corr[corr.cc > multicollinearity_threshold]        

    to_drop = []
    for f0, f1 in corr.index:
        if (f0 == f1) | any(feat in [f0, f1] for feat in to_drop):
            continue
        to_drop.append(pv.loc[[f0, f1]].sort_values(by='p_value', 
                                                ascending=False).index[0])
    x_cols = list(set(x_cols) - set(to_drop))
    return x_cols   

## Initial Model

Initial model using only cleaned data:

In [250]:
data = pd.read_csv('data/clean.csv')
data.date = pd.to_datetime(data.date)

data_train, data_test = tts(data, train_size=0.8, random_state=111)

In [251]:
model, x_cols = modelling(data_train, multicoll=multicoll_remove)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                     3201.
Date:                Tue, 17 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:24:59   Log-Likelihood:            -2.3283e+05
No. Observations:               16987   AIC:                         4.657e+05
Df Residuals:                   16976   BIC:                         4.658e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      3.374e+07   3.35e+06     10.081      0.000    2.72e+07    4.03e+07
view           6.146e+04   2583.138     23.794      0.000    5.64e+04    6.65e+04
long          -6.319e+04   1.53e+04     -4.125      0.000   -9.32e+04   -3.32e+04
bathrooms      1.133e+05   3251.522     34.832      0.000    1.07e+05     1.2e+05
grade          1.709e+05   2028.491     84.234      0.000    1.67e+05    1.75e+05
lat             5.78e+05    1.3e+04     44.515      0.000    5.53e+05    6.03e+05
sqft_basement    51.7196      4.356     11.872      0.000      43.181      60.258
yr_built      -3359.6118     80.017    -41.986      0.000   -3516.454   -3202.769
condition      1.829e+04   2800.976      6.530      0.000    1.28e+04    2.38e+04
zipcode        -646.7394     39.456    -16.391      0.000    -724.077    -569.402
waterfront     6.776e+05   2.16e+04     31.392      0.000    6.35e+05     7.2e+05
==============================================================================
Omnibus:                    15544.127   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1785880.400
Skew:                           3.993   Prob(JB):                         0.00
Kurtosis:                      52.592   Cond. No.                     1.97e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [252]:
model, x_cols = modelling(data_train)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.700
Method:                 Least Squares   F-statistic:                     2481.
Date:                Tue, 17 Nov 2020   Prob (F-statistic):               0.00
Time:                        14:25:02   Log-Likelihood:            -2.3159e+05
No. Observations:               16987   AIC:                         4.632e+05
Df Residuals:                   16970   BIC:                         4.633e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      1.309e+07   3.27e+06      3.999      0.000    6.67e+06    1.95e+07
bedrooms      -4.172e+04   2230.985    -18.699      0.000   -4.61e+04   -3.73e+04
bathrooms      4.427e+04   3597.383     12.307      0.000    3.72e+04    5.13e+04
sqft_living      96.2041     20.521      4.688      0.000      55.981     136.427
waterfront     6.636e+05   2.01e+04     32.932      0.000    6.24e+05    7.03e+05
view           5.128e+04   2443.710     20.986      0.000    4.65e+04    5.61e+04
condition      2.687e+04   2657.168     10.113      0.000    2.17e+04    3.21e+04
grade          9.641e+04   2454.954     39.271      0.000    9.16e+04    1.01e+05
sqft_above       93.4993     20.467      4.568      0.000      53.382     133.616
sqft_basement    64.3188     20.413      3.151      0.002      24.308     104.330
yr_built      -3232.0176    141.714    -22.807      0.000   -3509.792   -2954.243
yr_renovated    574.1015    145.970      3.933      0.000     287.986     860.217
zipcode        -627.3845     36.934    -16.987      0.000    -699.779    -554.990
lat            6.052e+05   1.21e+04     49.816      0.000    5.81e+05    6.29e+05
long          -1.982e+05   1.52e+04    -13.023      0.000   -2.28e+05   -1.68e+05
sqft_living15    21.1323      3.945      5.357      0.000      13.400      28.864
sqft_lot15       -1.3133      0.169     -7.769      0.000      -1.645      -0.982
==============================================================================
Omnibus:                    13803.971   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1108358.870
Skew:                           3.379   Prob(JB):                         0.00
Kurtosis:                      41.991   Cond. No.                     2.09e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""